In [6]:
import networkx as nx 
import pandas as pd
import numpy as np
import scipy.sparse
import time 
from random import *
import plotly.express as px

import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import inv

import scipy.sparse as sp

import tqdm


## Part 1

In [243]:
def meanTimeBFS(g, n, simSize):
    vec = []
    starting_node = randint(0, n-1)
    for x in range(simSize):
        #starting time
        t0 = time.time()

        #Create BFS Tree
        bfsTree = nx.bfs_tree(g, starting_node)

        #Check equality of graphs

        #ending time
        t1 = time.time()

        total = t1-t0
        vec.append(total)
    mean = sum(vec)/len(vec)
    return mean
    

In [244]:
def meanTimeEigen(g, simSize):
    vec = []
    for x in range(simSize):
        t0 = time.time()
        matrix = nx.laplacian_matrix(g)

        e = np.linalg.eigvals(matrix.A)

        #check for connectivity
        np.sort(e)[1] > 0
        
        t1 = time.time()
        total = t1-t0
        vec.append(total)
    mean = sum(vec)/len(vec)
    return mean

In [257]:

def meanTimeIrr(g, simSize):
    vec = []
    for x in range(simSize):
        t0 = time.time()
        adj_matrix = nx.to_numpy_array(g)

        final_A = np.identity(len(adj_matrix)) + adj_matrix # I + A

        A_tot = adj_matrix
        for _ in range(2, len(adj_matrix)): # A^2 + ... + A^(n-1)
            A_tot = A_tot.dot(adj_matrix)
            A_tot[A_tot != 0] = 1 # avoid overflow
            final_A = final_A + A_tot

        #check connectivity
        (final_A > 0).all() 

        t1 = time.time()
        total = t1-t0
        vec.append(total)
    mean = sum(vec)/len(vec)
    return mean

In [259]:
data = pd.DataFrame()
ERtimeBFS = []
ERtimeIrr = []
ERtimeEigen = []
RtimeBFS = []
RtimeIrr = []
RtimeEigen = []
for N in tqdm.tqdm(np.arange(10,250,10)):
   
    ER_G = nx.erdos_renyi_graph(N, 0.1 )
    R_G = nx.random_regular_graph(2, N)
    
    #BFS
    ERtimeBFS.append(meanTimeBFS(ER_G, N, 10))
    RtimeBFS.append(meanTimeBFS(R_G, N, 10))

    #Adjacency matrix
    ERtimeIrr.append(meanTimeIrr(ER_G, 10))
    RtimeIrr.append(meanTimeIrr(R_G, 10))
    
    #Eigen of Laplacian Matrix
    ERtimeEigen.append(meanTimeEigen(ER_G, 10))
    RtimeEigen.append(meanTimeEigen(R_G, 10))

data["ER_BFS"] =  pd.DataFrame(ERtimeBFS)
data["R_BFS"] = pd.DataFrame(RtimeBFS)
data["ER_EigenLaplacian"] = pd.DataFrame(ERtimeEigen)
data["R_EigenLaplacian"] = pd.DataFrame(RtimeEigen)
data["ER_Irr"] = pd.DataFrame(ERtimeIrr)
data["R_Irr"] = pd.DataFrame(RtimeIrr)

data = data.set_index(np.arange(10,250,10))
    

100%|██████████| 24/24 [00:20<00:00,  1.18it/s]


In [260]:
data.head()

ER_BFS  R_BFS  ER_EigenLaplacian  R_EigenLaplacian    ER_Irr     R_Irr
10  0.000109    0.0           0.001820          0.000760  0.000204  0.000000
20  0.000888    0.0           0.001608          0.002042  0.000760  0.000000
30  0.000823    0.0           0.001708          0.001588  0.000519  0.000809
40  0.000800    0.0           0.001389          0.001182  0.000924  0.000870
50  0.000000    0.0           0.001226          0.000800  0.002682  0.002330

In [263]:
fig = px.line(data[["ER_BFS", "ER_EigenLaplacian", "ER_Irr"]], 
    title = "Comparison of different methods for Connectivity with ER-Graphs",
    labels={
                     "index": "Number of Nodes",
                     "value": "mean time",
                     "variable": "Connectivity methods"
                 })
fig.show()

In [264]:
fig = px.line(data[["R_BFS", "R_EigenLaplacian","R_Irr"]], 
    title = "Comparison of different methods for Connectivity with R-Random-Graphs",
    labels={
                     "index": "Number of Nodes",
                     "value": "mean time",
                     "variable": "Connectivity methods"
                 })
fig.show()

### Part 1: Monte Carlo Simulation

### ER-Graphs

#### k = 100, p is changing

In [ ]:
#setup for the montecarlo
data2 = pd.DataFrame()
#fixed value for the nodes
K = 100
vec = []
#probability of ER graphs, growing by 0.1 on each iteration
for p in tqdm.tqdm(np.arange(0,1,0.005)):
    tempVec = []
    #for every probability we will do 1000 simulations
    for  i in range(200):
        ER_G = nx.erdos_renyi_graph(K, p)
        if nx.is_connected(ER_G):
            tempVec.append(1)
        else:
            tempVec.append(0)
    mean = sum(tempVec)/len(tempVec)
    vec.append(mean)


100%|██████████| 200/200 [05:45<00:00,  1.73s/it]


In [ ]:
data3 = pd.DataFrame()
data3["100"] =  pd.DataFrame(vec)
data3 = data3.set_index(np.arange(0,1,0.005))

In [207]:
data3

100
0.000  0.0
0.005  0.0
0.010  0.0
0.015  0.0
0.020  0.0
...    ...
0.975  1.0
0.980  1.0
0.985  1.0
0.990  1.0
0.995  1.0

[200 rows x 1 columns]

In [272]:
fig = px.line(data3[["100"]], 
    title = "Probability of ER-Graph being connected",
    labels = {
        "index": "p",
        "value": "probability of ER-Graph being connected",
        "variable": "N. of Nodes"
    })
fig.show()

### R-Graphs

#### r = 2 and r= 8


In [273]:
#setup for the montecarlo
data2 = pd.DataFrame()
#fixed value for the nodes
K = 100
vec2 = []
vec3 = []
#probability of ER graphs, growing by 0.1 on each iteration
for k in tqdm.tqdm(np.arange(10,101,5)):
    #for every probability we will do 1000 simulations
    
    tempVec = []
    tempVec2 = []
    for  i in range(200):
        
        #creo primo grafo con degree 2
        R_G2 = nx.random_regular_graph(2, k)

        #creo secondo grafo con degree 8
        R_G8 = nx.random_regular_graph(8, k)

        #check connectivity of graph with degree 2
        if nx.is_connected(R_G2):
            tempVec.append(1)
        else:
            tempVec.append(0)

        #check connectivity of graph with degree 8
        if nx.is_connected(R_G8):
            tempVec2.append(1)
        else:
            tempVec2.append(0)

    mean = sum(tempVec)/len(tempVec)
    vec2.append(mean) 
    
    mean = sum(tempVec2)/len(tempVec2)
    vec3.append(mean) 

100%|██████████| 19/19 [00:08<00:00,  2.22it/s]


In [275]:
data2 = pd.DataFrame()
data2["2"] =  pd.DataFrame(vec2)
data2["8"] =  pd.DataFrame(vec3)
data2 = data2.set_index(np.arange(10,101,5))

In [276]:
data2.head()

2    8
10   0.520  1.0
15   0.420  1.0
20   0.390  1.0
25   0.345  1.0
30   0.270  1.0
35   0.260  1.0
40   0.275  1.0
45   0.220  1.0
50   0.205  1.0
55   0.195  1.0
60   0.245  1.0
65   0.190  1.0
70   0.215  1.0
75   0.195  1.0
80   0.180  1.0
85   0.145  1.0
90   0.130  1.0
95   0.165  1.0
100  0.160  1.0

In [277]:
fig = px.line(data2[["2", "8"]], 
    title = "Probability of R-Graph being connected",
    labels={
                     "index": "Number of Nodes",
                     "value": "probability of R-Graph being connected",
                     "variable": "k"
                 })
fig.show()

## Part 2

 First we set up the variables for the simulation, that where give to us by the assignment

In [334]:
#SetUp
C = 1.25 #10/8
T0 = 30
Lo = 4*1024
Lf = 4*1024
tau = 5e-6

E_x = 8*60*60
epsilon = 0.1
f = 48/1500
n = 64

As stated in the assignment we also need to normalize the output by the R_baseline, which is the mean time in case of just the server A being used, and we will calculate it right now.

In [335]:
R_baseline = T0 + E_x
R_baseline

28830

## Mean time with Simulations

### Fat-Tree

In [397]:

avg_response_times1 = []
Ti_tot = 0
k = 0
for N in tqdm.tqdm(np.arange(1,10001)):
    times = []
    
    #hop size based on number of servers in the structure
    h = int(((np.log(N)//np.log((32))) + 1)*2)

    #value fo Ti
    Ti = 2*tau*h
    Ti_tot = Ti_tot + 1/Ti
    k = k + 1/N

    #throughput for each Server
    throughput = C * ((1/Ti)/Ti_tot) if N > 1 else C

    La = []

    Ttransmto = []
    Ttransmfrom = []

    #sim size for each Server Size
    for i in range(1,1000):

        #sampling from an exponential distribution for
        Xi = np.mean(np.random.exponential( scale = (E_x/N)))

        #time for the computation job
        La.append(k*Xi)

        #output of each Server
        Lo_i = np.random.uniform(0,2*Lo/N)

        #quando devi inviare al server
        Ttransmto.append(((Lf/N) + (f*(Lf/N)))/throughput)

        #quando il server deve inviare al server originale A
        Ttransmfrom.append(((Lo_i)*N/(N+1))/throughput)

    times = np.mean(La) if N == 1 else T0 + np.mean(La) + (np.mean(Ttransmto) + np.mean(Ttransmfrom))
    
    avg_response_times1.append(times)



100%|██████████| 10000/10000 [06:00<00:00, 27.73it/s]


### Jellyfish

In [400]:
avg_response_times2 = []

Ti_tot = 0
k = 0
for N in tqdm.tqdm(np.arange(1,10001)):
    times = []
    #hop size based on number of servers in the structure
    h = 2 if N < 32 else 3
    if N >= 1024:
        h = 4

    #value fo Ti
    Ti = 2*tau*h
    Ti_tot = Ti_tot + 1/Ti
    k = k + 1/N

    #throughput for each Server
    throughput = C * ((1/Ti)/Ti_tot)  if N > 1 else C

    La = []
    Ttransmto = []
    Ttransmfrom = []

    #sim size for each Server Size
    for i in range(1,1000):

        #sampling from an exponential distribution for
        Xi = np.mean(np.random.exponential(scale = (E_x/N)))

        #time for the computation job
        La.append(k*Xi)

        #output of each Server
        Lo_i = np.random.uniform(0,2*Lo/N)
    
        Ttransmto.append(((Lf/N) + (f*(Lf/N)))/throughput)
        Ttransmfrom.append(((Lo_i)*N/(N+1))/throughput)
        
    times = np.mean(La) if N == 1 else T0 +np.mean(La) + (np.mean(Ttransmto) + np.mean(Ttransmfrom))
    
    avg_response_times2.append(np.mean(times))

100%|██████████| 10000/10000 [05:14<00:00, 31.75it/s]


In [401]:
data4 = pd.DataFrame()
data4["jellyfish_mean"] = pd.DataFrame([ round(elem,3) for elem in avg_response_times2])
data4["fatTree_mean"] = pd.DataFrame([ round(elem,3) for elem in avg_response_times1])
data4 = data4.set_index(np.arange(1,10001))
data4["Jellyfish_mean_normalized"] = round(data4.jellyfish_mean/R_baseline, 3)
data4["fatTree_mean_normalized"] = round(data4.fatTree_mean/R_baseline,3)

In [402]:
data4.head()

jellyfish_mean  fatTree_mean  Jellyfish_mean_normalized  \
1       30387.593     29127.480                      1.054   
2       28392.481     27782.231                      0.985   
3       24089.322     23271.395                      0.836   
4       21957.849     20555.496                      0.762   
5       19238.132     18770.383                      0.667   

   fatTree_mean_normalized  
1                    1.010  
2                    0.964  
3                    0.807  
4                    0.713  
5                    0.651

In [403]:
fig = px.line(data4[["Jellyfish_mean_normalized", "fatTree_mean_normalized"]], 
    title = "Mean response time with respect to N",
    labels={
                     "index": "Number of Servers",
                     "value": "mean response time",
                     "variable": "Topology",
                     "Jellyfish_mean_normalized": "Jellyfish",
                     "fatTree_mean_normalized": "Fat Tree"
                 },
    color_discrete_sequence=px.colors.qualitative.Vivid)


fig.add_hline(y = 1, 
              line_width=3, 
              line_dash="dash", 
              line_color="red",
              annotation_text="R_baseline",
              annotation_position="top left")


fig.show()

## Mean time analytical 

### Fat Tree

In [390]:
avg_response_times3 = []
Ti_tot = 0
k = 0
for N in tqdm.tqdm(np.arange(1,10001)):
    times = []
    
    #hop size based on number of servers in the structure
    h = int(((np.log(N)//np.log((32))) + 1)*2)

    #value fo Ti
    Ti = 1/(2*tau*h)
    Ti_tot = Ti_tot + Ti
    k = k + 1/N

    #throughput for each Server
    throughput = C*(Ti/Ti_tot) 

    #sampling from an exponential distribution for
    Xi = E_x/N

    #time for the computation job
    La = k*Xi + T0

    #sum(1/n for n in range(1,N+1))
    #quando devi inviare al server
    timeTo = (((Lf/N) + (f*(Lf/N))))

    #quando il server deve inviare al server originale A
    timeFrom = (Lo/N)+ f*(Lo/N)

    
    times = La if N == 1 else La + (timeTo/throughput) + (timeFrom/throughput)
        
    
    avg_response_times3.append(times)

100%|██████████| 10000/10000 [00:00<00:00, 46641.73it/s]


### Jellyfish

In [391]:
avg_response_times4 = []
Ti_tot = 0
k = 0
for N in tqdm.tqdm(np.arange(1,10001)):
    times = []
    
    #hop size based on number of servers in the structure
    h = 2 if N < 32 else 3
    if N >= 1024:
        h = 4

    #value fo Ti
    Ti = 1/(2*tau*h)
    Ti_tot = Ti_tot + Ti

    k = k + 1/N

    #throughput for each Server
    throughput = C*(Ti/Ti_tot) 

    #sampling from an exponential distribution for
    Xi = E_x/N

    #time for the computation job
    La = k*Xi + T0

    #quando devi inviare al server
    timeTo = (((Lf/N) + (f*(Lf/N))))

    #quando il server deve inviare al server originale A
    timeFrom = (Lo/N)+ f*(Lo/N)

    
    times = La if N == 1 else La + (timeTo/throughput) + (timeFrom/throughput)
        
    
    avg_response_times4.append(times)

100%|██████████| 10000/10000 [00:00<00:00, 130170.97it/s]


In [392]:
data5 = pd.DataFrame()
data5["jellyfish_mean"] = pd.DataFrame([ elem for elem in avg_response_times4])
data5["fatTree_mean"] = pd.DataFrame([ elem for elem in avg_response_times3])
data5 = data5.set_index(np.arange(1,10001))
data5["Jellyfish_mean_normalized"] = data5.jellyfish_mean/R_baseline
data5["fatTree_mean_normalized"] = data5.fatTree_mean/R_baseline

In [393]:
data5.head()

jellyfish_mean  fatTree_mean  Jellyfish_mean_normalized  \
1      28830.0000    28830.0000                   1.000000   
2      28393.3152    28393.3152                   0.984853   
3      24393.3152    24393.3152                   0.846109   
4      21793.3152    21793.3152                   0.755925   
5      19945.3152    19945.3152                   0.691825   

   fatTree_mean_normalized  
1                 1.000000  
2                 0.984853  
3                 0.846109  
4                 0.755925  
5                 0.691825

In [394]:
fig = px.line(data5[["Jellyfish_mean_normalized", "fatTree_mean_normalized"]], 
    title = "Mean response time with respect to N",
    width=600, height=350,
    labels={
                     "index": "Number of Servers",
                     "value": "mean response time",
                     "variable": "Topology",
                     "Jellyfish_mean_normalized": "Jellyfish",
                     "fatTree_mean_normalized": "Fat Tree"
                 },
    color_discrete_sequence=px.colors.qualitative.Vivid)


fig.add_hline(y = 1, 
              line_width=3, 
              line_dash="dash", 
              line_color="red",
              annotation_text="R_baseline",
              annotation_position="top left")


fig.show()

## Job Cost with Simulations

As for the mean time, we need to normalize our result by the value of S_baseline which is the base case in which only the server A is used, and as with the previous R_baseline, we will find that value in the beggining

In [404]:
#In case of the Fat Tree
S_baseline = R_baseline + epsilon * data4[['fatTree_mean_normalized']].iloc[0,:]*R_baseline

#In case of the Jellyfish
S_baseline = R_baseline + epsilon * data4[['fatTree_mean_normalized']].iloc[0,:]*R_baseline

In [405]:
def costFunctionFatTree(N):
    epsilon = 0.1
    T0 = 30
    E_x = 8*60*60
    #In case of the Jellyfish
    S_baseline = R_baseline + epsilon * float(data4[['fatTree_mean_normalized']].iloc[0,:])*R_baseline


    #for the baseline
    listForN = []
    E_N = float(data4[['fatTree_mean_normalized']].iloc[N-1,:])*R_baseline
    #for x in range(1000):

    sum = 0
    for x in range(100):
        sum = sum + float(np.sum(np.random.exponential(size = N, scale = (E_x/N))))

    time = E_N + epsilon * ((N * T0) + sum/100)
    
    return(time/(S_baseline))

In [406]:
def costFunctionJellyfish(N):
    epsilon = 0.1
    T0 = 30
    E_x = 8*60*60

    
    #In case of the Jellyfish
    S_baseline = R_baseline + epsilon * float(data4[['Jellyfish_mean_normalized']].iloc[0,:])*R_baseline


    listForN = []
    E_N = float(data4[['Jellyfish_mean_normalized']].iloc[N-1,:])*R_baseline
    #for x in range(1000):

    sum = 0
    for x in range(100):
        sum = sum + float(np.sum(np.random.exponential(size = N, scale = (E_x/N))))

    time = E_N + epsilon * ((N * T0) + sum/100)
    
    
    return(time/(S_baseline))

Once we created the 2 functions, we will apply it for every N, with the data previosly collected

In [407]:
data4["Job_Cost_Jellyfish"] = data4.index.map(costFunctionJellyfish)
data4["Job_Cost_FatTree"] = data4.index.map(costFunctionFatTree)
data4["Job_Cost_Jellyfish_r"] = round(data4.Job_Cost_Jellyfish,3)
data4["Job_Cost_FatTree_r"] = round(data4.Job_Cost_FatTree,3)

In [408]:
data4.head()

jellyfish_mean  fatTree_mean  Jellyfish_mean_normalized  \
1       30387.593     29127.480                      1.054   
2       28392.481     27782.231                      0.985   
3       24089.322     23271.395                      0.836   
4       21957.849     20555.496                      0.762   
5       19238.132     18770.383                      0.667   

   fatTree_mean_normalized  Job_Cost_Jellyfish  Job_Cost_FatTree  \
1                    1.010            1.045757          1.011830   
2                    0.964            0.987193          0.970964   
3                    0.807            0.842603          0.829933   
4                    0.713            0.787657          0.744958   
5                    0.651            0.694731          0.685719   

   Job_Cost_Jellyfish_r  Job_Cost_FatTree_r  
1                 1.046               1.012  
2                 0.987               0.971  
3                 0.843               0.830  
4                 0.788               0.745  
5                 0.695               0.686

In [409]:
fig = px.line(data4[["Job_Cost_Jellyfish", "Job_Cost_FatTree"]], 
    title = "Job cost with respect to N",
    labels={
                     "index": "Number of Servers",
                     "value": "Job Cost",
                     "variable": "Topology",
                     "Jellyfish_mean_normalized": "Jellyfish",
                     "fatTree_mean_normalized": "Fat Tree"
                 },
    color_discrete_sequence=px.colors.qualitative.Vivid)


fig.add_hline(y = 1, 
              line_width=3, 
              line_dash="dash", 
              line_color="red",
              annotation_text="S_baseline",
              annotation_position="top left")
fig.show()

## Job Cost analytical

In [379]:
def costFunctionFatTreeA(N):
    epsilon = 0.1
    T0 = 30
    E_x = 8*60*60
    #In case of the Jellyfish
    S_baseline = R_baseline + epsilon * float(data5[['fatTree_mean_normalized']].iloc[0,:])*R_baseline


    #for the baseline
    listForN = []
    E_N = float(data5[['fatTree_mean_normalized']].iloc[N-1,:])*R_baseline
    #for x in range(1000):

    time = E_N + epsilon * ((N * T0) + E_x)
    
    return(time/(S_baseline))

In [380]:
def costFunctionJellyfishA(N):
    epsilon = 0.1
    T0 = 30
    E_x = 8*60*60

    
    #In case of the Jellyfish
    S_baseline = R_baseline + epsilon * float(data5[['Jellyfish_mean_normalized']].iloc[0,:])*R_baseline


    listForN = []
    E_N = float(data5[['Jellyfish_mean_normalized']].iloc[N-1,:])*R_baseline
    #for x in range(1000):

    time = E_N + epsilon * ((N * T0) + E_x)
    
            
    return(time/(S_baseline))

In [381]:
#write everything inside a dataFrame
data5["Job_Cost_Jellyfish"] = data5.index.map(costFunctionJellyfishA)
data5["Job_Cost_FatTree"] = data5.index.map(costFunctionFatTreeA)
data5["Job_Cost_Jellyfish_r"] = round(data5.Job_Cost_Jellyfish,3)
data5["Job_Cost_FatTree_r"] = round(data5.Job_Cost_FatTree,3)

In [382]:
fig = px.line(data5[["Job_Cost_Jellyfish", "Job_Cost_FatTree"]], 
    title = "Job cost with respect to N",
    width=600, height=350,
    labels={
                     "index": "Number of Servers",
                     "value": "Job Cost",
                     "variable": "Topology",
                     "Jellyfish_mean_normalized": "Jellyfish",
                     "fatTree_mean_normalized": "Fat Tree"
                 },
    color_discrete_sequence=px.colors.qualitative.Vivid)


fig.add_hline(y = 1, 
              line_width=2, 
              line_dash="dash", 
              line_color="red",
              annotation_text="S_baseline",
              annotation_position="top left")
fig.show()

## Find the optimal number of servers to use to optimize the job running cost

To find the optimal number of servers in our case we just need to select the minimum number obtained from the "Job_cost_fatTree" and the "Job Cost_Jellyfish"

So for the jellyfish we will have:

In [ ]:
data5.loc[data5['Job_Cost_Jellyfish'] == min(data5.Job_Cost_Jellyfish)]

jellyfish_mean  fatTree_mean  Jellyfish_mean_normalized  \
292     7769.325451   8128.337045                   0.269488   

     fatTree_mean_normalized  Job_Cost_Jellyfish  Job_Cost_FatTree  \
292                  0.28194            0.363426          0.374747   

     Job_Cost_Jellyfish_r  Job_Cost_FatTree_r  
292                 0.363               0.375

and for the fat tree instead we have

In [ ]:
data5.loc[data5['Job_Cost_FatTree'] == min(data5.Job_Cost_FatTree)]

jellyfish_mean  fatTree_mean  Jellyfish_mean_normalized  \
349     7624.611788   7924.988251                   0.264468   

     fatTree_mean_normalized  Job_Cost_Jellyfish  Job_Cost_FatTree  \
349                 0.274887            0.364255          0.373726   

     Job_Cost_Jellyfish_r  Job_Cost_FatTree_r  
349                 0.364               0.374

In [ ]:
float(data5[['jellyfish_mean']].iloc[0,:])

28830.0

Lets's visualize the obtimal number of servers in more detail

In [ ]:
fig = px.line(data5.iloc[0:1500,4:6], 
    title = "Job cost with respect to N",
    width=600, height=350,
    labels={
                     "index": "Number of Servers",
                     "value": "Job Cost",
                     "variable": "Topology",
                     "Jellyfish_mean_normalized": "Jellyfish",
                     "fatTree_mean_normalized": "Fat Tree"
                 },
    color_discrete_sequence=px.colors.qualitative.Vivid)
fig.add_vline(x= 292, 
              line_width=2, 
              line_dash="dash", 
              line_color="orange",
              annotation_text="jellyfish optimal",
              annotation_position="bottom left")

fig.add_vline(x= 349, 
              line_width=2, 
              line_dash="dash", 
              line_color="blue",
              annotation_text="fat tree optimal",
              annotation_position="bottom right")


fig.add_hline(y = 1, 
              line_width=2, 
              line_dash="dash", 
              line_color="red",
              annotation_text="S_baseline",
              annotation_position="top left")
fig.show()

In [348]:
data4.to_pickle("my_data")

In [ ]:
data4 = pd.read_pickle("my_data")